### QnA with GraphDB

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os

In [3]:
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [4]:
from langchain_community.graphs import Neo4jGraph

In [5]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

In [6]:
graph

In [7]:
movie_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
m.title = row.title,
m.imdbRating = toFloat(row.imdbRating)

FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))

FOREACH (actor in split(row.actors, '|') |
    MERGE (p: Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))

FOREACH (genre in split(row.genres, '|') |
    MERGE (g: Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))

"""

In [8]:
movie_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE (m:Movie {id:row.movieId})\nSET m.released = date(row.released),\nm.title = row.title,\nm.imdbRating = toFloat(row.imdbRating)\n\nFOREACH (director in split(row.director, '|') |\n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\n\nFOREACH (actor in split(row.actors, '|') |\n    MERGE (p: Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\n\nFOREACH (genre in split(row.genres, '|') |\n    MERGE (g: Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n"

In [9]:
graph.query(movie_query)

[]

In [10]:
print(graph.schema)

Node properties are the following:

Relationship properties are the following:

The relationships are the following:



In [11]:
graph.refresh_schema()

In [12]:
print(graph.schema)

Node properties are the following:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT},Person {name: STRING},Genre {name: STRING}
Relationship properties are the following:

The relationships are the following:
(:Movie)-[:IN_GENRE]->(:Genre),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)


In [13]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [14]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=groq_api_key, model_name="gemma2-9b-it")

In [15]:
from langchain.chains import GraphCypherQAChain

In [ ]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

In [17]:
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000001F84B062050>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F85F51B0D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F85F654B50>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))

In [19]:
response = chain.invoke({"query":"Who was the director of the movie Casino?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name  

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


In [20]:
response

{'query': 'Who was the director of the movie Casino?',
 'result': 'Martin Scorsese  \n'}

In [23]:
response = chain.invoke({"query":"Who acted in the movie Casino?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:ACTED_IN]-(p:Person) RETURN p.name
Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.


In [24]:
response

{'query': 'Who acted in the movie Casino?',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods acted in the movie Casino.  \n'}

In [25]:
response = chain.invoke({"query":"How many artists are there in total?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN count(p) 

Full Context:
[{'count(p)': 1239}]

> Finished chain.


In [26]:
response

{'query': 'How many artists are there in total?',
 'result': 'There are 1239 artists in total. \n'}

In [27]:
response = chain.invoke({"query":"How many movies does Tom Hanks acted in?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie)
RETURN COUNT(m)

Full Context:
[{'COUNT(m)': 2}]

> Finished chain.


In [28]:
response

{'query': 'How many movies does Tom Hanks acted in?',
 'result': "I don't know the answer. \n"}